### Configuración y Preparación del Entrenamiento de Red Neuronal en PyTorch con TIMM y Torchvision

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import h5py
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from timm import create_model
from torchinfo import summary
from tqdm import tqdm

# Configuracion
DATA_DIR = "C:/Fernando/Temas C/PROYECTO/Pytorch - copia"
BATCH_SIZE = 32
NUM_EPOCHS = 40
LEARNING_RATE = 0.0001
TRAIN_SPLIT = 0.75
VAL_SPLIT = 0.2
DIM = (64,64)

### Configuración del Dispositivo, Transformaciones de Datos y División en Conjuntos de Entrenamiento, Validación y Prueba en PyTorch

In [3]:
# Cuda train the model with GPU, hence we have to enable the GPU of our workstation
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformations
transform = transforms.Compose([
    transforms.Resize(DIM),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load dataset
dataset = datasets.ImageFolder(root=DATA_DIR, transform=transform)
num_classes = len(dataset.classes)

# Split dataset
train_size = int(TRAIN_SPLIT*len(dataset))
val_size = int(VAL_SPLIT*len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

### Implementación del Bucle de Entrenamiento y Evaluación del Modelo con PyTorch, Scheduler y Guardado de Pesos

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import h5py
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

# Training loop
def train_model(model,
                train_loader,
                val_loader,
                criterion,
                optimizer,
                num_epochs=NUM_EPOCHS,
                nameSave="modelo"):

    best_val_loss = float("inf")
    history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

    # 🔹 Agregar un Scheduler para reducir el LR si la pérdida de validación no mejora
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=3, verbose=True)

    for epoch in range(num_epochs):
        model.train()
        train_loss, correct, total = 0, 0, 0

        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)
        for images, labels in loop:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            loop.set_postfix(train_loss=train_loss / total, train_acc=correct / total)

        train_loss /= total
        train_acc = correct / total
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)

        # 🔹 Validation loop
        model.eval()
        val_loss, correct, total = 0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_loss /= total
        val_acc = correct / total
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(f"Epoch {epoch+1}/{num_epochs} || Train Loss: {train_loss:.4f} || Train Acc: {train_acc:.4f} || Val Loss: {val_loss:.4f} || Val Acc: {val_acc:.4f}")

        # 🔹 Guardar el mejor modelo
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f"{nameSave}.pth")
            with h5py.File(f"{nameSave}.h5", "w") as f:
                for name, param in model.state_dict().items():
                    f.create_dataset(name, data=param.cpu().numpy())
            print("✅ Model SAVED in H5 format")

        # 🔹 Reducir LR si la pérdida de validación no mejora
        scheduler.step(val_loss)

    return history  # 🔹 Devuelve el historial de entrenamiento

# Test evaluation
def evaluate_model(model, test_loader, criterion, nameSave):
    # Crear una instancia del modelo antes de cargar los pesos
    model = CustomCNN(num_classes=num_classes).to(DEVICE)  # Asegúrate de que la arquitectura es la correcta
    model.load_state_dict(torch.load(f"{nameSave}.pth", map_location=DEVICE))
    model.eval()  # Ahora sí se puede usar eval()
    
    test_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    test_loss /= total
    test_acc = correct / total
    print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

### Arquitectura de la Red Neuronal Convolucional (CNN) Personalizada para Clasificación de Imágenes

Descripción:
- Normalización LayerNorm para estabilizar la entrada.
- Cinco capas convolucionales con BatchNorm y ReLU para la extracción de características.
- Adaptive Average Pooling para reducir la dimensionalidad antes de la capa totalmente conectada.
- Tres capas totalmente conectadas (FC) con dropout para prevenir overfitting.
- Función de pérdida CrossEntropyLoss y optimizador Adam con decaimiento del peso.
- Scheduler de tasa de aprendizaje (StepLR) para reducir la tasa cada 10 épocas.

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import h5py
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler
from torchvision import datasets, transforms, models
import numpy as np
from collections import Counter
from torchinfo import summary
from tqdm import tqdm

nameSaved = "ModelAvarageV8"
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()

        self.norm = nn.LayerNorm([3, DIM[0], DIM[1]], elementwise_affine=False)
        self.conv1 = nn.Sequential(nn.Conv2d(3, 16, kernel_size=5, stride=2), nn.BatchNorm2d(16), nn.ReLU())
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=5, stride=2), nn.BatchNorm2d(32), nn.ReLU())
        self.conv3 = nn.Conv2d(32,64, kernel_size=5, stride=2)
        self.conv4 = nn.Conv2d(64,128, kernel_size=3)
        self.conv5 = nn.Conv2d(128,256, kernel_size=3)

        self.gap = nn.AdaptiveAvgPool2d(1)
        self.flater = nn.Flatten()

        self.fc1 = nn.Linear(256,128)
        self.dropout = nn.Dropout(0.3)  # 🔹 Previene overfitting
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,3)

    def forward(self, x):
        x = self.norm(x)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = torch.relu(self.conv5(x))
        x = self.gap(x)
        x = self.flater(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)

        return x

# Iniciar cnn
model1 = CustomCNN(num_classes=num_classes).to(DEVICE)

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model1.parameters(), lr=0.001, weight_decay=1e-4)
  # 🔹 Menos penalización
# Scheduler de Learning Rate
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)


# Resumen
summary(model1, input_size=(1,3,DIM[0],DIM[1]))

Layer (type:depth-idx)                   Output Shape              Param #
CustomCNN                                --                        --
├─LayerNorm: 1-1                         [1, 3, 64, 64]            --
├─Sequential: 1-2                        [1, 16, 30, 30]           --
│    └─Conv2d: 2-1                       [1, 16, 30, 30]           1,216
│    └─BatchNorm2d: 2-2                  [1, 16, 30, 30]           32
│    └─ReLU: 2-3                         [1, 16, 30, 30]           --
├─Sequential: 1-3                        [1, 32, 13, 13]           --
│    └─Conv2d: 2-4                       [1, 32, 13, 13]           12,832
│    └─BatchNorm2d: 2-5                  [1, 32, 13, 13]           64
│    └─ReLU: 2-6                         [1, 32, 13, 13]           --
├─Conv2d: 1-4                            [1, 64, 5, 5]             51,264
├─Conv2d: 1-5                            [1, 128, 3, 3]            73,856
├─Conv2d: 1-6                            [1, 256, 1, 1]            295

In [ ]:
train_model(model1, train_loader, val_loader, criterion, optimizer, nameSave=nameSaved)

In [ ]:
evaluate_model(model1, test_loader, criterion=criterion, nameSave=nameSaved)